In [352]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime

In [353]:
Fecha_Inicio = "2023-08-01"

In [354]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)

In [355]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
    }
)
Pacientes_SF.shape

(1809645, 2)

In [356]:
nuevos = "F:/Procesos Automaticos/AccuHealth/1.xlsx"
Archivo = pd.read_excel(nuevos,sheet_name='Accuhealth')

# OJO se copia rut1 a rut le saca el guoion
Archivo.head()

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,Medicacion,SBP
0,25785,9512321,06060944-6,Juan Guillermo,Bernstein,2023-01-02T10:05:32.000Z,Baja,117,Bien,75,Sí,106
1,25785,9515540,06060944-6,Juan Guillermo,Bernstein,2023-01-04T08:45:32.000Z,Baja,133,Bien,70,Sí,109
2,25785,9519833,06060944-6,Juan Guillermo,Bernstein,2023-01-06T08:45:43.000Z,Baja,128,Bien,74,Sí,118
3,25785,9524222,06060944-6,Juan Guillermo,Bernstein,2023-01-09T08:45:05.000Z,Alta,97,Bien,79,Sí,113
4,25785,9525982,06060944-6,Juan Guillermo,Bernstein,2023-01-09T20:45:06.000Z,Incompleto,Observación faltante,Bien,Observación faltante,Sí,Observación faltante


In [357]:
Archivo["RUT"] = Archivo["RUT"].astype(str)
Archivo["RUT"] = Archivo["RUT"].str.replace("-", "")
Archivo["RUT"] = Archivo["RUT"].str.replace(".", "")
Archivo["RUT"] = Archivo["RUT"].str.upper()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12584\3527352582.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Archivo["RUT"] = Archivo["RUT"].str.replace(".", "")


In [358]:
Archivo1.dtypes

patient_lot_id                  int64
Rut                            object
Fecha                          object
Fecha_chilena          datetime64[ns]
severity                       object
BG                             object
bienestar                      object
Medicacion                     object
Id Account Paciente            object
Presión Arterial               object
dtype: object

In [359]:
def siPrimerCero(y):
    if y[0] == "0":
        y = y[1:]
    return y

In [360]:
Archivo["RUT2"] = Archivo["RUT"].apply(lambda y: siPrimerCero(y))
Archivo.head()

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,Medicacion,SBP,RUT2
0,25785,9512321,060609446,Juan Guillermo,Bernstein,2023-01-02T10:05:32.000Z,Baja,117,Bien,75,Sí,106,60609446
1,25785,9515540,060609446,Juan Guillermo,Bernstein,2023-01-04T08:45:32.000Z,Baja,133,Bien,70,Sí,109,60609446
2,25785,9519833,060609446,Juan Guillermo,Bernstein,2023-01-06T08:45:43.000Z,Baja,128,Bien,74,Sí,118,60609446
3,25785,9524222,060609446,Juan Guillermo,Bernstein,2023-01-09T08:45:05.000Z,Alta,97,Bien,79,Sí,113,60609446
4,25785,9525982,060609446,Juan Guillermo,Bernstein,2023-01-09T20:45:06.000Z,Incompleto,Observación faltante,Bien,Observación faltante,Sí,Observación faltante,60609446


In [361]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12584\2920035971.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")


In [362]:
Archivo = pd.merge(
    left=Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="RUT2",
    right_on="Rut",
)
Archivo.head()

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,Medicacion,SBP,RUT2,Id Account Paciente,Rut
0,25785,9512321,060609446,Juan Guillermo,Bernstein,2023-01-02T10:05:32.000Z,Baja,117,Bien,75,Sí,106,60609446,0018c00002amPjDAAU,60609446
1,25785,9515540,060609446,Juan Guillermo,Bernstein,2023-01-04T08:45:32.000Z,Baja,133,Bien,70,Sí,109,60609446,0018c00002amPjDAAU,60609446
2,25785,9519833,060609446,Juan Guillermo,Bernstein,2023-01-06T08:45:43.000Z,Baja,128,Bien,74,Sí,118,60609446,0018c00002amPjDAAU,60609446
3,25785,9524222,060609446,Juan Guillermo,Bernstein,2023-01-09T08:45:05.000Z,Alta,97,Bien,79,Sí,113,60609446,0018c00002amPjDAAU,60609446
4,25785,9525982,060609446,Juan Guillermo,Bernstein,2023-01-09T20:45:06.000Z,Incompleto,Observación faltante,Bien,Observación faltante,Sí,Observación faltante,60609446,0018c00002amPjDAAU,60609446


In [363]:
Archivo["Presión Arterial"] = np.where(Archivo["DBP"].str.len() <=4,Archivo["DBP"] + " / " + Archivo["SBP"],"0")

Archivo.head()

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,Medicacion,SBP,RUT2,Id Account Paciente,Rut,Presión Arterial
0,25785,9512321,060609446,Juan Guillermo,Bernstein,2023-01-02T10:05:32.000Z,Baja,117,Bien,75,Sí,106,60609446,0018c00002amPjDAAU,60609446,75 / 106
1,25785,9515540,060609446,Juan Guillermo,Bernstein,2023-01-04T08:45:32.000Z,Baja,133,Bien,70,Sí,109,60609446,0018c00002amPjDAAU,60609446,70 / 109
2,25785,9519833,060609446,Juan Guillermo,Bernstein,2023-01-06T08:45:43.000Z,Baja,128,Bien,74,Sí,118,60609446,0018c00002amPjDAAU,60609446,74 / 118
3,25785,9524222,060609446,Juan Guillermo,Bernstein,2023-01-09T08:45:05.000Z,Alta,97,Bien,79,Sí,113,60609446,0018c00002amPjDAAU,60609446,79 / 113
4,25785,9525982,060609446,Juan Guillermo,Bernstein,2023-01-09T20:45:06.000Z,Incompleto,Observación faltante,Bien,Observación faltante,Sí,Observación faltante,60609446,0018c00002amPjDAAU,60609446,0


In [364]:
Archivo["Dia"] = pd.to_datetime(Archivo["date"]).dt.day

Archivo["Mes"] = pd.to_datetime(Archivo["date"]).dt.month

Archivo["Mes Nombre"] = pd.to_datetime(Archivo["date"]).dt.month_name()

Archivo["Año"] = pd.to_datetime(Archivo["date"]).dt.year

Archivo["Fecha_chilena"] = pd.to_datetime(
    Archivo["date"], dayfirst=True, yearfirst=False, exact=True
)
# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
# Archivo['LocalDateTime'] = pd.to_datetime(Archivo['LocalDateTime'])
# 

Archivo["Fecha_chilena"] = Archivo["Fecha_chilena"].apply(
    lambda a: datetime.strftime(a, "%Y-%m-%d %H:%M:%S")
)

Archivo["Fecha_chilena"] = pd.to_datetime(Archivo["Fecha_chilena"])

Archivo["Fecha"] = Archivo["Fecha_chilena"].dt.strftime("%d-%m-%Y")
Archivo = Archivo[(Archivo["severity"] != "No realizado")]

Discriminacion fecha datos relevantes para carga

In [365]:
Archivo1 = Archivo[(Archivo["Fecha_chilena"] >= Fecha_Inicio)]
Archivo1.sort_values(by="date")

,patient_id,patient_lot_id,RUT,name,surname,date,severity,BG,bienestar,DBP,...,RUT2,Id Account Paciente,Rut,Presión Arterial,Dia,Mes,Mes Nombre,Año,Fecha_chilena,Fecha
3235,31878,9901128,146301541,Eduardo,De la Rosa Rebaza,2023-08-01T00:29:24.000Z,Baja,123,Bien,NaN,...,146301541,0018c00002amTvkAAE,146301541,0,1,8,August,2023,2023-08-01 00:29:24,01-08-2023
422,29224,9901131,072492277,Guillermo Felipe,Arce Fuentes,2023-08-01T00:56:12.000Z,Baja,NaN,Bien,NaN,...,72492277,0018c00002amLvLAAU,72492277,0,1,8,August,2023,2023-08-01 00:56:12,01-08-2023
423,29224,9901132,072492277,Guillermo Felipe,Arce Fuentes,2023-08-01T00:57:33.000Z,Moderada,200,NaN,80,...,72492277,0018c00002amLvLAAU,72492277,80 / 126,1,8,August,2023,2023-08-01 00:57:33,01-08-2023
3775,32888,9901142,112571132,Danahe,Méndez Valera,2023-08-01T05:32:20.000Z,Moderada,167,Bien,NaN,...,112571132,0018c00002amSFzAAM,112571132,0,1,8,August,2023,2023-08-01 05:32:20,01-08-2023
9123,37509,9901145,120892029,Genoveva,Díaz González,2023-08-01T05:40:03.000Z,Moderada,NaN,Bien,84,...,120892029,0018c00002ao1y6AAA,120892029,84 / 116,1,8,August,2023,2023-08-01 05:40:03,01-08-2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10254,38315,9952789,131492766,Cristian,Gangas Matus,2023-08-28T23:00:07.000Z,Moderada,NaN,Bien,78,...,131492766,0018c00002ao2N2AAI,131492766,78 / 111,28,8,August,2023,2023-08-28 23:00:07,28-08-2023
447,29224,9952833,072492277,Guillermo Felipe,Arce Fuentes,2023-08-28T23:36:44.000Z,Moderada,189,NaN,66,...,72492277,0018c00002amLvLAAU,72492277,66 / 103,28,8,August,2023,2023-08-28 23:36:44,28-08-2023
5082,38313,9952834,162310445,Andrés,Silva Veas,2023-08-28T23:41:43.000Z,Baja,95,Bien,NaN,...,162310445,0018c00002bKW89AAG,162310445,0,28,8,August,2023,2023-08-28 23:41:43,28-08-2023
7588,35812,9952846,116359235,Juan,Briones Paredes,2023-08-28T23:59:36.000Z,Moderada,NaN,Bien,66,...,116359235,0018c00002amLciAAE,116359235,66 / 136,28,8,August,2023,2023-08-28 23:59:36,28-08-2023


In [366]:
Archivo1 = Archivo1[
    [
        "patient_lot_id",
        "Rut",
        "Fecha",
        "Fecha_chilena",
        "severity",
        "BG",
        "bienestar",
        "Medicacion",
        "Id Account Paciente",
        "Presión Arterial",
    ]
]
Archivo1.shape

(994, 10)

In [372]:
results = sf.query_all(
    """
    select Id,idAccuHealth__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c 
   """
)
fichaAccu = pd.DataFrame(results["records"])

fichaAccu = fichaAccu[
    [
        "Id",
        "idAccuHealth__c",
    ]
]
fichaAccu = fichaAccu.rename(
    columns={
        "Id": "Id Account Paciente",
        "idAccuHealth__c": "Codigo",
    }
)
fichaAccu.shape

(2956, 2)

In [373]:
fichaAccu.head()

,Id Account Paciente,Codigo
0,a1f8c000001ovZuAAI,None
1,a1f8c000001ovZzAAI,None
2,a1f8c000001ova4AAA,None
3,a1f8c000001ova9AAA,None
4,a1f8c000001ovaEAAQ,None


In [374]:
fichaAccu["Codigo"] = fichaAccu["Codigo"].astype(str)


In [375]:

Archivo1["Estaono"] = Archivo1["patient_lot_id"].isin(fichaAccu["Codigo"])

Archivo1.head()

,patient_lot_id,Rut,Fecha,Fecha_chilena,severity,BG,bienestar,Medicacion,Id Account Paciente,Presión Arterial,Estaono
184,9927929,112296999,16-08-2023,2023-08-16 06:18:08,Moderada,152,Bien,No,0018c00002amLvfAAE,86 / 131,False
185,9932225,112296999,18-08-2023,2023-08-18 06:30:46,Moderada,132,Bien,No,0018c00002amLvfAAE,75 / 118,False
186,9936593,112296999,21-08-2023,2023-08-21 06:22:39,Moderada,132,Bien,No,0018c00002amLvfAAE,84 / 138,False
187,9940895,112296999,23-08-2023,2023-08-23 06:11:17,Moderada,137,Bien,No,0018c00002amLvfAAE,78 / 126,False
188,9945113,112296999,25-08-2023,2023-08-25 06:21:17,Moderada,132,Bien,Sí,0018c00002amLvfAAE,77 / 123,False
